Generative and DL techniques on small metabolic engineering datasets

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pio.renderers.default = "vscode"

def get_X_y(df: pd.DataFrame, target: str = 'prod') -> (pd.DataFrame, pd.Series):
    X = df.drop(columns=[target])
    y = df[target]
    return X, y

def get_train_test(X: pd.DataFrame, y: pd.Series, test_size: float = 0.2) -> (pd.DataFrame, pd.DataFrame, pd.Series, pd.Series):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

def normalize(scaler: StandardScaler, X_train: pd.DataFrame, X_test: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    X_train_norm = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_norm =  pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    return X_train_norm, X_test_norm

__Get data__

In [2]:
df = pd.read_csv('./data/limonene_production.csv', index_col=0)
print(f' * Dataset shape: {df.shape} *')

 * Dataset shape: (30, 10) *


In [3]:
X, y = get_X_y(df, 'Limonene')

X_train, X_test, y_train, y_test = get_train_test(X, y)

X_train_norm, X_test_norm = normalize(StandardScaler(), X_train, X_test)

__Model Selection__

In [32]:
#  Tune and train an SVR using RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_distributions = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': uniform(0.0001, 10),
    'gamma': uniform(0.0001, 0.1),
    'epsilon': uniform(0.1, 1.0)
}


svr = SVR()
rnd_search = RandomizedSearchCV(svr, param_distributions, n_iter=100, cv=5, verbose=2, n_jobs=-1)
rnd_search.fit(X_train_norm, y_train)

print(f' * Best parameters: {rnd_search.best_params_} *')
print(f' * Best score: {rnd_search.best_score_} *')

model = rnd_search.best_estimator_


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END C=9.69657230869307, epsilon=0.7054785609768681, gamma=0.05503473987487026, kernel=linear; total time=   0.0s
[CV] END C=9.69657230869307, epsilon=0.7054785609768681, gamma=0.05503473987487026, kernel=linear; total time=   0.0s
[CV] END C=9.711338455516126, epsilon=0.3755338663984378, gamma=0.08167997196825329, kernel=linear; total time=   0.0s
[CV] END C=9.69657230869307, epsilon=0.7054785609768681, gamma=0.05503473987487026, kernel=linear; total time=   0.0s
[CV] END C=9.711338455516126, epsilon=0.3755338663984378, gamma=0.08167997196825329, kernel=linear; total time=   0.0s
[CV] END C=9.69657230869307, epsilon=0.7054785609768681, gamma=0.05503473987487026, kernel=linear; total time=   0.0s
[CV] END C=9.69657230869307, epsilon=0.7054785609768681, gamma=0.05503473987487026, kernel=linear; total time=   0.0s
[CV] END C=8.04205835037789, epsilon=0.6616013964626615, gamma=0.0005870017846887787, kernel=rbf; total time=

In [33]:
# Validate the model using the test set
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

y_pred = model.predict(X_test_norm)

results = pd.DataFrame(data={
    'Actual': y_test,
    'Predicted': y_pred
})

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f' * Mean Squared Error: {mse} *')
print(f' * R2 Score: {r2} *')
print(f' * Mean Absolute Error: {mae} *')

 * Mean Squared Error: 344.8547599382114 *
 * R2 Score: 0.37394638025947746 *
 * Mean Absolute Error: 14.738020182206773 *


### Synthetic Data Vault (SDV)

In [8]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(X)

Gaussian Copula Synthesizer

In [10]:
from sdv.single_table import GaussianCopulaSynthesizer

gaussian_copula = GaussianCopulaSynthesizer(metadata=metadata)
gaussian_copula.fit(X)
gaussian_copula_synthetic_data = gaussian_copula.sample(1000)

Copula GAN Synthesizer

In [11]:
from sdv.single_table import CopulaGANSynthesizer

copula_gan = CopulaGANSynthesizer(metadata=metadata)
copula_gan.fit(X)
copula_gan_synthetic_data = copula_gan.sample(1000)

TVAE Synthesizer

In [12]:
from sdv.single_table import TVAESynthesizer

tvaesynthesizer = TVAESynthesizer(metadata=metadata)
tvaesynthesizer.fit(X)
tvaesynthesizer_synthetic_data = tvaesynthesizer.sample(1000)

CTGAN Synthesizer

In [14]:
from sdv.single_table import CTGANSynthesizer

ctgan = CTGANSynthesizer(metadata=metadata)
ctgan.fit(X)
ctgan_synthetic_data = ctgan.sample(1000)

### Statistical Validation

In [15]:
fake_data_df = ctgan_synthetic_data.copy()

In [16]:
from sdmetrics.reports.single_table import DiagnosticReport

diagnostic = DiagnosticReport()
diagnostic.generate(X, fake_data_df, metadata.to_dict())

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 8/8 [00:00<00:00, 913.02it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 244.85it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%



In [19]:
from sdmetrics.reports.single_table import QualityReport

quality_report = QualityReport()
quality_report.generate(X, fake_data_df, metadata.to_dict())

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 8/8 [00:00<00:00, 728.18it/s]|
Column Shapes Score: 77.37%

(2/2) Evaluating Column Pair Trends: |██████████| 28/28 [00:00<00:00, 174.76it/s]|
Column Pair Trends Score: 90.83%

Overall Score (Average): 84.1%



In [21]:
from sdmetrics.visualization import get_column_plot

for c in X.columns:
    fig = get_column_plot(X, fake_data_df, c)
    pio.renderers.default = "vscode"
    fig.show()
    break

In [20]:
pp = quality_report.get_visualization('Column Pair Trends')
pio.renderers.default = "vscode"
pp

In [22]:
from sdmetrics.single_table import NewRowSynthesis

NewRowSynthesis.compute(
    real_data=X,
    synthetic_data=fake_data_df,
    metadata=metadata
)

1.0

### Machine Learning Detection Model

In [30]:
real_data = X.copy()
synthetic_data = fake_data_df.copy()

real_data['cls'] = 0
synthetic_data['cls'] = 1

synthetic_data = synthetic_data.sample(real_data.shape[0])

print(f' * Real data shape: {real_data.shape} *')
print(f' * Synthetic data shape: {synthetic_data.shape} *')

 * Real data shape: (168, 9) *
 * Synthetic data shape: (168, 9) *


In [33]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
data = pd.concat([real_data, synthetic_data], axis=0)

X_ml = data.drop('cls', axis=1)
y_ml = data['cls']

X_train, X_test, y_train, y_test = train_test_split(X_ml, y_ml, test_size=0.33, random_state=42)

lr.fit(X_train, y_train)
lr.score(X_test, y_test)



0.5495495495495496

### GANBRLR

In [38]:
from ganblr.models import GANBLR

ganblr = GANBLR()
ganblr.fit(X, y, k=0, epochs=10, batch_size=32)

warmup run:
6/6 [==============================] - 1s 11ms/step - loss: 5.1243 - accuracy: 0.0119


/home/WUR/g0012069/env/main/lib/python3.10/site-packages/ganblr/models/ganblr.py:76: RuntimeWarning:

divide by zero encountered in log



Epoch 1/10: G_loss = inf, G_accuracy = 0.035714, D_loss = 30.627460, D_accuracy = 0.552239
